In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime as dt
import ast

In [23]:
demo_cleaned = pd.read_table(r'../data/demo_cleaned.csv', sep = ',')
exp_cleaned = pd.read_table(r'../data/exp_cleaned.csv', sep = ',')
web_cleaned = pd.read_table(r'../data/web_cleaned.csv', sep = ',')
step_fail_df = pd.read_table(r'../data/step_fail_df.csv', sep = ',')
step_time_delta = pd.read_table(r'../data/step_time_delta.csv', sep = ',')
backward_total_df = pd.read_table(r'../data/backward_total_df.csv', sep = ',')

In [2]:
step_time_delta = pd.read_csv(r'../data/step_time_delta.csv', sep = ',')

In [24]:
test_join = web_cleaned.merge(exp_cleaned[['client_id', 'Variation']], on='client_id', how='left', suffixes=('', '_exp'), validate='many_to_one')

In [25]:
web_test = test_join[test_join['Variation'] == 'Test']
web_control = test_join[test_join['Variation'] == 'Control']

## Count backward steps and compare

In [26]:
# Create the shift dfs
web_test_shift = web_test.shift(1)
web_control_shift = web_control.shift(1)

backward_step = [('confirm','step_3'),('confirm','step_2'),('confirm','step_1'),('confirm','start'),
                 ('step_3','step_2'),('step_3','step_1'),('step_3','start'),
                 ('step_2','step_1'),('step_2','start'),
                 ('step_1','start')]

# For the test group
web_test.loc[:, 'backward_step'] = [step in backward_step for step in zip(web_test['process_step'], web_test_shift['process_step'])]
count_backward_steps_test = web_test['backward_step'].sum()

# For the control group
web_control.loc[:, 'backward_step'] = [step in backward_step for step in zip(web_control['process_step'], web_control_shift['process_step'])]
count_backward_steps_control = web_control['backward_step'].sum()

C:\Users\matth\AppData\Local\Temp\ipykernel_16072\2816313643.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  web_test.loc[:, 'backward_step'] = [step in backward_step for step in zip(web_test['process_step'], web_test_shift['process_step'])]
C:\Users\matth\AppData\Local\Temp\ipykernel_16072\2816313643.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  web_control.loc[:, 'backward_step'] = [step in backward_step for step in zip(web_control['process_step'], web_control_shift['process_step'])]


In [27]:
backward_dict = {'control' : {'count': count_backward_steps_control, 'percentage': round(count_backward_steps_control/len(web_control['client_id'])*100,2)},
                 'test': {'count': count_backward_steps_test, 'percentage': round(count_backward_steps_test/len(web_test['client_id'])*100,2)}}

backward_total_df = pd.DataFrame.from_dict(backward_dict, orient='index')

In [28]:
# Identify fail step in Test and Control

web_test['fail_step'] = np.where(web_test['backward_step'], web_test['process_step'], 'NA')

web_control['fail_step'] = np.where(web_control['backward_step'], web_control['process_step'], 'NA')


C:\Users\matth\AppData\Local\Temp\ipykernel_16072\1230023161.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  web_test['fail_step'] = np.where(web_test['backward_step'], web_test['process_step'], 'NA')
C:\Users\matth\AppData\Local\Temp\ipykernel_16072\1230023161.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  web_control['fail_step'] = np.where(web_control['backward_step'], web_control['process_step'], 'NA')


In [29]:
# Create df for individual step fails
test_step_fails_df = pd.DataFrame(web_test['fail_step'].value_counts())
test_step_fails_df['percentage'] = round(test_step_fails_df['count']/len(web_test['client_id'])*100,2)

control_step_fails_df = pd.DataFrame(web_control['fail_step'].value_counts())
control_step_fails_df['percentage'] = round(control_step_fails_df['count']/len(web_control['client_id'])*100,2)

In [56]:
# Create merged df for both trials
step_fail_df = pd.merge(control_step_fails_df, test_step_fails_df, how = 'inner', left_index = True, right_index = True, suffixes = ('_control','_test'))
step_fail_df.sort_index(inplace=True)

In [45]:
# Get totals of each step to calculate pct fail
tuple_dict_test = web_test['step_tuple'].value_counts().to_dict()
tuple_dict_control = web_control['step_tuple'].value_counts().to_dict()

## Fail rates

Having removed false repeated steps - which might contain automatic refreshes or loss of user connection - it is possible to count the quantity of "backward steps" - i.e. points in the process where users were forced to reverse direction in their journey. These can also be broken down into individual steps whereby analysts may observe the particular steps that account for process failures or delays. 

In [62]:
backward_total_df

,count,percentage
control,29879,26.33
test,38675,27.39


In [55]:
step_fail_df

,count_control,percentage_control,count_test,percentage_test
fail_step,,,,
NA,83615,73.67,102518,72.61
confirm,13967,12.31,17653,12.50
step_1,5244,4.62,8116,5.75
step_2,3771,3.32,5924,4.20
step_3,6897,6.08,6982,4.95


## Time taken

This process will group the data by visitor_id and sum the length of time spent on each step.

In [31]:
# Cast date_time column to datetime in pandas
dataframes = [web_control, web_control_shift, web_test, web_test_shift]
for df in dataframes:
    df['date_time'] = pd.to_datetime(df['date_time'])

# Create delta time column using shift(1) and assign steps from each column as new tuple column
web_control = web_control.assign(
    delta_time = (web_control_shift['date_time'] - web_control['date_time']).shift(-1),
    step_tuple = list(zip(web_control['process_step'], web_control_shift['process_step']))
)

web_test = web_test.assign(
    delta_time = web_test_shift['date_time'] - web_test['date_time'],
    step_tuple = list(zip(web_test['process_step'], web_test_shift['process_step']))
)

# Shift step_tuple column to match delta_time
web_control['step_tuple'] = web_control['step_tuple'].shift(-1)
web_test['step_tuple'] = web_test['step_tuple'].shift(-1)

C:\Users\matth\AppData\Local\Temp\ipykernel_16072\3654970805.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date_time'] = pd.to_datetime(df['date_time'])
C:\Users\matth\AppData\Local\Temp\ipykernel_16072\3654970805.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date_time'] = pd.to_datetime(df['date_time'])


In [136]:
# Create new df filtering out any delta_time values over 1 hour and negative values
control_delta_drop = web_control[((web_control['delta_time'] >= '0 days 01:00:00') | (web_control['delta_time'] <= '0 days 00:00:00'))].index
test_delta_drop = web_test[(web_test['delta_time'] >= '0 days 01:00:00') | (web_test['delta_time'] <= '0 days 00:00:00')].index

# Drop values over 1 hour and any negative values
control_delta_df = web_control.drop(index = control_delta_drop)
test_delta_df = web_test.drop(index = test_delta_drop)

# Group by step type and sum
control_step_sum = control_delta_df.groupby('step_tuple')['delta_time'].sum()
test_step_sum = test_delta_df.groupby('step_tuple')['delta_time'].sum()

In [159]:
# Reset index
# control_step_sum = control_step_sum.reset_index(drop=False)

# Split 'step_tuple' into two new columns 'step_start' and 'step_end'
control_step_sum[['step_start', 'step_end']] = pd.DataFrame(control_step_sum['step_tuple'].tolist(), index=control_step_sum.index)

control_step_sum = control_step_sum.drop(columns=['level_0','index'])

In [165]:
# Reset index
# test_step_sum = test_step_sum.reset_index(drop=False)

# Split 'step_tuple' into two new columns 'step_start' and 'step_end'
test_step_sum[['step_start', 'step_end']] = pd.DataFrame(test_step_sum['step_tuple'].tolist(), index=test_step_sum.index)

test_step_sum = test_step_sum.drop(columns='index')

In [170]:
# Create merged df

step_time_delta = pd.merge(control_step_sum, test_step_sum, on = ['step_tuple','step_start','step_end'], how = 'inner', suffixes = ('_control','_test'))

In [179]:
# Create dictionary to assign numerical values to steps
order_dict = {'start':0,'step_1':1,'step_2':2,'step_3':3,'confirm':4}

# Map dictionary to new columns for sorting
step_time_delta['step_start_v'] = step_time_delta['step_start'].map(order_dict)
step_time_delta['step_end_v'] = step_time_delta['step_end'].map(order_dict)

In [184]:
step_time_delta.columns

Index(['step_tuple', 'delta_time_control', 'step_start', 'step_end',
       'delta_time_test', 'step_start_v', 'step_end_v'],
      dtype='object')

In [188]:
# Sort and reorder columns
step_time_delta = step_time_delta.sort_values(by = ['step_start_v','step_end_v']).reset_index(drop=True)
step_time_delta = step_time_delta[['step_tuple','step_start_v','step_end_v','step_start','step_end',
      'delta_time_control','delta_time_test']]
step_time_delta

,step_tuple,step_start_v,step_end_v,step_start,step_end,delta_time_control,delta_time_test
0,"(start, start)",0,0,start,start,4 days 16:35:11,3 days 14:02:16
1,"(start, step_1)",0,1,start,step_1,10 days 02:31:42,23 days 01:34:48
2,"(start, step_2)",0,2,start,step_2,1 days 00:16:38,1 days 23:56:24
3,"(start, step_3)",0,3,start,step_3,1 days 01:14:07,0 days 19:43:30
4,"(start, confirm)",0,4,start,confirm,0 days 07:06:43,0 days 12:33:13
5,"(step_1, start)",1,0,step_1,start,7 days 01:34:54,4 days 06:35:18
6,"(step_1, step_1)",1,1,step_1,step_1,0 days 13:46:50,0 days 10:09:06
7,"(step_1, step_2)",1,2,step_1,step_2,8 days 05:05:12,26 days 23:43:20
8,"(step_1, step_3)",1,3,step_1,step_3,1 days 12:45:31,1 days 16:36:57
9,"(step_1, confirm)",1,4,step_1,confirm,0 days 15:39:24,0 days 08:46:30


In [14]:
step_time_delta['delta_time_control'] = pd.to_timedelta(step_time_delta['delta_time_control'])
step_time_delta['delta_time_test'] = pd.to_timedelta(step_time_delta['delta_time_test'])

In [15]:
step_time_delta = step_time_delta.assign(total_seconds_control = step_time_delta['delta_time_control'].dt.total_seconds(), 
                                         total_seconds_test = step_time_delta['delta_time_test'].dt.total_seconds())

In [83]:
# Convert 'step_tuple' to tuple type
step_time_delta['step_tuple'] = step_time_delta['step_tuple'].apply(ast.literal_eval)

# map the dictionary values
step_time_delta['avg_seconds_control'] = step_time_delta['total_seconds_control'] / step_time_delta['step_tuple'].map(tuple_dict_control)
step_time_delta['avg_seconds_test'] = step_time_delta['total_seconds_test'] / step_time_delta['step_tuple'].map(tuple_dict_test)


In [84]:
step_time_delta

,Unnamed: 0.1,Unnamed: 0,step_tuple,step_start_v,step_end_v,step_start,step_end,delta_time_control,delta_time_test,total_seconds_control,total_seconds_test,avg_seconds_control,avg_seconds_test
0,0,0,"(start, start)",0,0,start,start,4 days 16:35:11,3 days 14:02:16,405311.0,309736.0,52.706242,41.430712
1,1,1,"(start, step_1)",0,1,start,step_1,10 days 02:31:42,23 days 01:34:48,873102.0,1992888.0,40.046876,70.659765
2,2,2,"(start, step_2)",0,2,start,step_2,1 days 00:16:38,1 days 23:56:24,87398.0,172584.0,68.980268,102.728571
3,3,3,"(start, step_3)",0,3,start,step_3,1 days 01:14:07,0 days 19:43:30,90847.0,71010.0,161.937611,96.089310
4,4,4,"(start, confirm)",0,4,start,confirm,0 days 07:06:43,0 days 12:33:13,25603.0,45193.0,69.762943,52.488966
5,5,5,"(step_1, start)",1,0,step_1,start,7 days 01:34:54,4 days 06:35:18,610494.0,369318.0,116.417620,45.504929
6,6,6,"(step_1, step_1)",1,1,step_1,step_1,0 days 13:46:50,0 days 10:09:06,49610.0,36546.0,132.293333,64.455026
7,7,7,"(step_1, step_2)",1,2,step_1,step_2,8 days 05:05:12,26 days 23:43:20,709512.0,2331800.0,38.364443,102.092820
8,8,8,"(step_1, step_3)",1,3,step_1,step_3,1 days 12:45:31,1 days 16:36:57,132331.0,146217.0,129.609207,119.948318
9,9,9,"(step_1, confirm)",1,4,step_1,confirm,0 days 15:39:24,0 days 08:46:30,56364.0,31590.0,189.777778,57.963303


In [82]:
# Get the first key in the dictionary
first_key = list(tuple_dict_control.keys())[0]

# Print the type of the first key
print(type(first_key))


<class 'tuple'>


## Exporting data

In [85]:
step_time_delta.to_csv('step_time_delta', sep = ',')

In [190]:
step_fail_df.to_csv('step_fail_df', sep = ',')

In [191]:
backward_total_df.to_csv('backward_total_df', sep = ',')